<a href="https://colab.research.google.com/github/cha0stooo/f16flyingdatasets/blob/master/lagrange_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls -R



.:
sample_data

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras import Model

class MyModel(Model):
  def __init__(self,layers):  
    super(MyModel, self).__init__()
    tf.keras.backend.set_floatx("float64")  
    self.layer_list = []
    self.bn = []
    
    for width in layers[1:-2]:
        self.layer_list.append(Dense(width, activation='relu',kernel_initializer=tf.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        self.bn.append(BatchNormalization())
    
    #self.layer_list.append(Dense(layers[-1], activation='linear',kernel_initializer= tf.initializers.normal()))        
    self.Lo_layer = Dense(layers[-2],activation='linear',kernel_initializer= 'glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01))
    self.Ld_layer = Dense(layers[-1],activation='relu',kernel_initializer=tf.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2(0.01))
     #定义一个ld的偏置
    self.bias_d = []
    bias_d = abs(tf.random.normal([1,3],dtype=tf.float64))
    self.bias_d.append(tf.Variable(bias_d, dtype=tf.float64, trainable=True))

  @tf.function
  def call(self, x):
    for index in range(len(self.layer_list)):
      x = self.layer_list[index](x)
      x = self.bn[index](x)
        
    lo = self.Lo_layer(x)
    ld = self.Ld_layer(x)+self.bias_d[0]
    
    return lo,ld
    
def save_model(model):
  #保存模型参数
  # checkpoint = tf.train.Checkpoint(myModel=model)
  # checkpoint.save(path+'/model.ckpt')
  model.save_weights('weight_tf_savedmodel_h5', save_format='h5')
    
def restore_model(model,path):
  #恢复模型参数
  checkpoint = tf.train.Checkpoint(myModel=model)
  checkpoint.restore(tf.train.latest_checkpoint(path))

#产生飞行器训练数据
import numpy as np 
import matplotlib.pyplot as plt
import time
np.random.seed(0)
tf.random.set_seed(0)


class vehicle_net():
  def __init__(self,layers,hp):
    self.epochs = hp["epochs"]
    self.tf_optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp["tf_lr"],
        beta_1=hp["tf_b1"],
        epsilon=hp["tf_eps"])
    self.LAM = hp["lamda"]
    #batchsize = hp['batchsize']
    self.model = MyModel(layers=layers)


  @tf.function
  def cal_inverse_dynamic(self,q,dq,ddq):
    q = tf.convert_to_tensor(q)
    dq = tf.convert_to_tensor(dq)   
    dq = tf.reshape(dq,[dq.shape[0],dq.shape[1],1])  #NX3 -> NX3X1
    ddq = tf.convert_to_tensor(ddq)
    ddq = tf.reshape(ddq,[ddq.shape[0],ddq.shape[1],1])  #NX3 -> NX3X1
    sample_num = q.shape[0]
    with tf.GradientTape(persistent=True) as tp:
      tp.watch(q) 
      Y_lo,Y_ld = self.model(q)   
      l1 = Y_ld[:,0:1]
      l2 = Y_ld[:,1:2]
      l3 = Y_ld[:,2:3]
      l4 = Y_lo[:,0:1]
      l5 = Y_lo[:,1:2]
      l6 = Y_lo[:,2:3]
      # #计算dLi_dq
      dL1_dq = tp.gradient(l1,q)  #NX3
      dL2_dq = tp.gradient(l2,q)  #NX3
      dL3_dq = tp.gradient(l3,q)  #NX3
      dL4_dq = tp.gradient(l4,q)  #NX3
      dL5_dq = tp.gradient(l5,q)  #NX3
      dL6_dq = tp.gradient(l6,q)  #NX3
      #计算dl_dqi
      dLd_dq1 = tf.stack([dL1_dq[:,0:1],dL2_dq[:,0:1],dL3_dq[:,0:1]],axis=1)  #NX3X1
      dLd_dq1 = tf.squeeze(dLd_dq1,axis=2)  #NX3
      dLd_dq2 = tf.stack([dL1_dq[:,1:2],dL2_dq[:,1:2],dL3_dq[:,1:2]],axis=1)
      dLd_dq2 = tf.squeeze(dLd_dq2,axis=2)  #NX3  
      dLd_dq3 = tf.stack([dL1_dq[:,2:3],dL2_dq[:,2:3],dL3_dq[:,2:3]],axis=1)
      dLd_dq3 = tf.squeeze(dLd_dq3,axis=2)  #NX3  
      dLo_dq1 = tf.stack([dL4_dq[:,0:1],dL5_dq[:,0:1],dL6_dq[:,0:1]],axis=1)
      dLo_dq1 = tf.squeeze(dLo_dq1,axis=2)  #NX3  
      dLo_dq2 = tf.stack([dL4_dq[:,1:2],dL5_dq[:,1:2],dL6_dq[:,1:2]],axis=1)  
      dLo_dq2 = tf.squeeze(dLo_dq2,axis=2)  #NX3
      dLo_dq3 = tf.stack([dL4_dq[:,2:3],dL5_dq[:,2:3],dL6_dq[:,2:3]],axis=1)  
      dLo_dq3 = tf.squeeze(dLo_dq3,axis=2)  #NX3
    del tp
    #计算逆动力学方程中的各项：M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
    #1.J = L*L'
    L_mat,L_mat_T,J = self.cal_J(Y_lo,Y_ld)
    #2.dJ/dt = dLdt@L'+L@dL'dt = dLdt@L'+L@dLdt'
    dL1_dq = tf.reshape(dL1_dq,[dL1_dq.shape[0],1,dL1_dq.shape[1]])
    dL2_dq = tf.reshape(dL2_dq,[dL2_dq.shape[0],1,dL2_dq.shape[1]])
    dL3_dq = tf.reshape(dL3_dq,[dL3_dq.shape[0],1,dL3_dq.shape[1]])
    dL4_dq = tf.reshape(dL4_dq,[dL4_dq.shape[0],1,dL4_dq.shape[1]])
    dL5_dq = tf.reshape(dL5_dq,[dL5_dq.shape[0],1,dL5_dq.shape[1]])
    dL6_dq = tf.reshape(dL6_dq,[dL6_dq.shape[0],1,dL6_dq.shape[1]])
    dl11 = tf.squeeze(dL1_dq@dq,axis=2)   #NX1X1 -> NX1
    dl22 = tf.squeeze(dL2_dq@dq,axis=2)
    dl33 = tf.squeeze(dL3_dq@dq,axis=2)
    dl21 = tf.squeeze(dL4_dq@dq,axis=2)
    dl31 = tf.squeeze(dL5_dq@dq,axis=2)
    dl32 = tf.squeeze(dL6_dq@dq,axis=2)
    dl12 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
    dl13 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
    dl23 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
    dl = tf.stack([dl11,dl12,dl13,dl21,dl22,dl23,dl31,dl32,dl33],axis=1)
    dl = tf.squeeze(dl,axis=2)
    dl_mat = tf.reshape(dl,[dl.shape[0],3,3])
    dl_mat_T = tf.transpose(dl_mat,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置 
    dJ = dl_mat@L_mat_T+L_mat@dl_mat_T
    #3.(dq'@J@dq)/dq
    #dLo_dq1 = dY_lo[:,:,0]     #NX3
    #dLd_dq1 = dY_ld[:,:,0]
    dl_dq1_mat = self.vec2mat(dLd_dq1,dLo_dq1)   #dl_dq1
    # dLo_dq2 = dY_lo[:,:,1]
    # dLd_dq2 = dY_ld[:,:,1]
    dl_dq2_mat = self.vec2mat(dLd_dq2,dLo_dq2)
    # dLo_dq3 = dY_lo[:,:,2]
    # dLd_dq3 = dY_ld[:,:,2]
    dl_dq3_mat = self.vec2mat(dLd_dq3,dLo_dq3)
    #计算dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq
    dq_T = tf.transpose(dq,perm=[0,2,1])    #NX3X1 ->NX1X3
    tmp1 = dq_T@(dl_dq1_mat@L_mat_T+L_mat@tf.transpose(dl_dq1_mat,perm=[0,2,1]))@dq   #NX1X1
    tmp2 = dq_T@(dl_dq2_mat@L_mat_T+L_mat@tf.transpose(dl_dq2_mat,perm=[0,2,1]))@dq   #NX1X1
    tmp3 = dq_T@(dl_dq3_mat@L_mat_T+L_mat@tf.transpose(dl_dq3_mat,perm=[0,2,1]))@dq   #NX1X1
    tmp = tf.stack([tmp1,tmp2,tmp3],axis=1)
    tmp = tf.squeeze(tmp,axis=3)   #NX3X1X1  -> NX3X1
    #-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]   #NX3X1
    f2 = -0.5*tmp
    #4.计算M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
    M_pre0 = J@ddq+f2+dJ@dq
    gama,phi = q[:,0:1],q[:,1:2]
    r11 = tf.ones_like(gama)
    r12 = tf.zeros_like(gama)
    r13 = -tf.sin(phi)
    r21 = tf.zeros_like(gama)
    r22 = tf.cos(gama)
    r23 = tf.sin(gama)*tf.cos(phi)
    r31 = tf.zeros_like(gama)
    r32 = -tf.sin(gama)
    r33 = tf.cos(gama)*tf.cos(phi)
    R = tf.stack([r11,r12,r13,r21,r22,r23,r31,r32,r33],axis=1)
    R = tf.squeeze(R,axis=2)
    R = tf.reshape(R,[R.shape[0],3,3])
    R_T = tf.transpose(R,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置
    M_pre = tf.linalg.inv(R_T)@M_pre0
    return M_pre0    #NX3X1

  @tf.function
  def vec2mat(self,dLd_dq1,dLo_dq1):
    #将N个3X1的dld_dqi 以及dlo_dqi 组合成 N个 3X3的矩阵
    dl_dq11 = dLd_dq1[:,0:1]
    dl_dq12 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)
    dl_dq13 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)
    dl_dq21 = dLo_dq1[:,0:1]
    dl_dq22 = dLd_dq1[:,1:2]
    dl_dq23 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)
    dl_dq31 = dLo_dq1[:,1:2]
    dl_dq32 = dLo_dq1[:,2:3]
    dl_dq33 = dLd_dq1[:,2:3]
    dl = tf.stack([dl_dq11,dl_dq12,dl_dq13,dl_dq21,dl_dq22,dl_dq23,dl_dq31,dl_dq32,dl_dq33],axis=1)
    dl = tf.squeeze(dl,axis=2)
    dl_dq_mat = tf.reshape(dl,[dl.shape[0],3,3])
    return dl_dq_mat
  @tf.function  
  def cal_J(self,Lo,Ld): 
    l1 = Ld[:,0:1]
    l2 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
    l3 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
    l4 = Lo[:,0:1]
    l5 = Ld[:,1:2]
    l6 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
    l7 = Lo[:,1:2]
    l8 = Lo[:,2:3]
    l9 = Ld[:,2:3]
    L_vec = tf.stack([l1,l2,l3,l4,l5,l6,l7,l8,l9],axis=1)
    L_vec = tf.squeeze(L_vec,axis=2)
    L_mat = tf.reshape(L_vec,[L_vec.shape[0],3,3])
    L_vec_T = tf.stack([l1,l4,l7,l2,l5,l8,l3,l6,l9],axis=1)
    L_vec_T = tf.squeeze(L_vec_T,axis=2)
    L_mat_T = tf.reshape(L_vec_T,[L_vec_T.shape[0],3,3])
    J_mat = L_mat@L_mat_T
    #J_vec_total = tf.reshape(J_mat,[J_mat.shape[0],9])
    return L_mat,L_mat_T,J_mat  
  @tf.function
  def train(self,q,dq,ddq,mom0):
    with tf.GradientTape() as tp:
      M_pre0 = self.cal_inverse_dynamic(q,dq,ddq)
      #M_pre = tf.squeeze(M_pre0,axis=2)
      #四种计loss方式等价
      #loss = (tf.keras.losses.MSE(M_pre[:,0],mom[:,0])\
          # +tf.keras.losses.MSE(M_pre[:,1],mom[:,1])\
          # +tf.keras.losses.MSE(M_pre[:,2],mom[:,2]))/3
      #loss = tf.reduce_mean(tf.keras.losses.MSE(mom,M_pre))
      res_loss = tf.reduce_mean(tf.keras.losses.MSE(mom0,M_pre0))
      loss = res_loss#+LAM*self.model.get_l2_loss()
      # loss0 = tf.norm(M_pre[:,0]-mom[:,0])**2/samp_num + \
      #     tf.norm(M_pre[:,1]-mom[:,1])**2/samp_num+\
      #         tf.norm(M_pre[:,2]-mom[:,2])**2/samp_num
    variables = self.model.trainable_variables
    grads = tp.gradient(loss,variables)
    self.tf_optimizer.apply_gradients(grads_and_vars=zip(grads,variables))
    del tp
    #print(epoch,loss.numpy())
    return loss


  def fit(self,q,dq,ddq,Mom,mini_batch_size=128):     
    loss_save = []
    N_data = q.shape[0]
    for ep in range(self.epochs): 
      #idx_data = np.random.choice(N_data, min(mini_batch_size, N_data))
      qi   = q#[idx_data,:]
      dqi  = dq#[idx_data,:]
      ddqi = ddq#[idx_data,:]
      Momi = Mom#[idx_data,:]
      Momi = tf.reshape(Momi,[Momi.shape[0],Momi.shape[1],1])
      time1 = time.time()                           
      loss = self.train(qi,dqi,ddqi,Momi)                
      loss_numpy = loss.numpy()
      loss_save.append(loss_numpy) 
      if ep%100==0:
          time2 = time.time()
          tt1 = time2-time1
          tf.print(tt1,ep,loss_numpy)
    return loss_save

from google.colab import files

def main():
  #1.数据准备
  state_roll = np.loadtxt('./f16flyingdatasets/state_roll2020-9-25.dat')
  state_pitch = np.loadtxt('./f16flyingdatasets/state_pitch2020-9-25.dat')
  state_yaw = np.loadtxt('./f16flyingdatasets/state_yaw2020-9-25.dat')

  state = np.vstack((state_roll,state_pitch,state_yaw)) 
  state = state 
  q = state[:,0:3]
  dq = state[:,3:6]
  ddq = state[:,6:9]
  mom = state[:,9:12]
  #mom0 = tf.reshape(mom,[mom.shape[0],mom.shape[1],1])
  #2.网络初始化
  layers = [3]+9*[9]+[3]+[3]
  hp = {}
  hp["epochs"] = 100000
  hp["tf_lr"] = 0.005
  hp["tf_b1"] = 0.9
  hp["tf_eps"] = 0.000000001
  hp["lamda"] = 0.01
  hp['batchsize'] = q.shape[0]
  vnet = vehicle_net(layers=layers,hp=hp)

  #3.训练
  losses = vnet.fit(q,dq,ddq,mom,mini_batch_size=q.shape[0])  
  #保存结果
  np.savetxt('./f16flyingdatasets/loss.dat',losses)
  #path = 'save/926'
  # vnet.model.save_model(path)
  #save_model(vnet.model, "./f16flyingdatasets/save/")
  save_model(vnet.model) 
  #restore_model(vnet.model,"saved/")
  files.download('weight_tf_savedmodel_h5')



def validate():
  import matplotlib.pyplot as plt
  #1.数据准备
  state_roll = np.loadtxt('./f16flyingdatasets/state_roll2020-9-16.dat')
  state_pitch = np.loadtxt('./f16flyingdatasets/state_pitch2020-9-16.dat')
  state_yaw = np.loadtxt('./f16flyingdatasets/state_yaw2020-9-16.dat')
  state = np.vstack((state_roll,state_pitch,state_yaw))
  state = state_roll
  q = state[:,0:3]
  dq = state[:,3:6]
  ddq = state[:,6:9]
  mom = state[:,9:12]
  mom = tf.reshape(mom,[mom.shape[0],mom.shape[1],1])
  #恢复网络
  hp = {}
  hp["epochs"] = 2000
  hp["tf_lr"] = 0.01
  hp["tf_b1"] = 0.9
  hp["tf_eps"] = 0.000000001
  hp["lamda"] = 0.01
  #hp["batchsize"] = 1001
  layers = [3]+12*[9]+[3]+[3]
  vnet = vehicle_net(layers=layers,hp=hp)

  vnet.model.load_weights('weight_tf_savedmodel_h5')
  #restore_model(vnet.model,"./f16flyingdatasets/save/")

  t1 = time.time()
  mom_pred = vnet.cal_inverse_dynamic(q,dq,ddq)
  t2 = time.time()
  print(t2-t1)
  plt.figure()
  #loss = np.loadtxt('loss.dat')
  #plt.plot(loss)  
  plt.figure()
  for fig_id in range(3):
      plt.subplot(3,1,fig_id+1)
      plt.plot(mom_pred[:,fig_id,:])
      plt.plot(mom[:,fig_id,:],'--')

  plt.show()
if __name__ == "__main__":
  main()
  #validate()


9.40610122680664 0 13674443596.645185
0.01847052574157715 100 207646958.41944325
0.018664121627807617 200 179491664.15446705
0.01863265037536621 300 175824510.58289298
0.01942920684814453 400 174313082.55906504
0.01866936683654785 500 173536675.77072683
0.018590450286865234 600 173132271.7160028
0.019902706146240234 700 172896929.4878741
0.018343687057495117 800 172753477.7789039
0.01852130889892578 900 172687839.61427695
0.01859140396118164 1000 172590832.27799666
0.018666982650756836 1100 172502395.77847406
0.0185394287109375 1200 172423275.52381682
0.01879096031188965 1300 172346353.71167186
0.018476247787475586 1400 172270720.46722436
0.01859116554260254 1500 172199582.79743204
0.01843428611755371 1600 172127477.48384982
0.018297910690307617 1700 172057947.00079447
0.018551349639892578 1800 171992024.48133036
0.023175477981567383 1900 171923605.0864993
0.01864337921142578 2000 171857422.67061606
0.018581628799438477 2100 171793687.6756429
0.018751144409179688 2200 171728027.5770331

In [2]:
!git clone https://github.com/cha0stooo/f16flyingdatasets.git


Cloning into 'f16flyingdatasets'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.
